# Code Supplement 2 for "On-Demand Public Transit: a Markovian Continuous Approximation Model"

### Daniel F. Silva
### Alexander Vinel
### Bekircan Kirkici

###### Department of Industrial and Systems Engineering, Auburn University

##### silva@auburn.edu
##### alexander.vinel@auburn.edu
##### bekir@auburn.edu

# Imports and Initial Globals

We import required packages and give an initial value for the global parameters we use.

In [2]:
import numpy as np
import pandas as pd
from collections import Counter
from datetime import datetime


In [1]:
carN = 2 # total number of cars 
depotCap = 3 # max customers in the depot #CHANGEABLE
carCap = 3 # max customers that can be in a car at any given time #DO NOT CHANGE
tH = 1 # at what number, does the car start to pick up from depot.
        # tH = 1: no wait, immediate pickup when at least 1 person is waiting
        # tH > 1: wait until tH people are in the depot waiting, to pick up.

L = 1        
M = [0.75, 0.44830504827348755, 0.3469597134667903] #


# All Functions

This section holds all the functions required to run. They should be run before producing any output.

### "active" and "eventspace"

_active_ : A binary function to control and show if we can move from state $s$ to $s'$, with a given event $e$.

_eventspace_ : A trivial function that outputs the event space of the Markov chain, with given number of cars $U$.

In [3]:
def active(e, s): #checks if a state movement is possible with the given event.   
    if s != None:
        #########
        for check in range(len(s)-1):
            if s[check] > carCap:
                return "car cap"
            elif s[len(s) - 1] > depotCap:
                return "depot cap"    
        #########
        if s != None:
            if e == 0:
                for i in range(len(s)):
                    if (s[i] == 0 or s[-1] < depotCap):
                        return 1
                    else:
                        return 0

            elif e > 0:
                if 0 < s[e-1] <= carCap:
                    return 1

                elif s[e-1] == 0 and s[-1] > tH:
                    return 1

                else:
                    return 0

def eventspace(carN = 1): # create the simple eventspace list 
    EventSpace = [i for i in range(carN + 1)]
    return EventSpace

### "new"

finds the first possible state to move from state $s'$, using current state $s$ with event $e$, starting from $e = 0, 1, .., U$

In [4]:
def new(e, sss): # this should return the new state
    if sss != None:   
    #cSet = Counter(s[0:-1]) 
    
    #########
        for check in range(len(sss)-1):
            if sss[check] > carCap:
                return "car cap"
            elif sss[len(sss) - 1] > depotCap:
                return "depot cap"
    #########

        s = sss.copy()
        if e == 0: # this is Lambda, thus arrival
            for i in range(len(s)): # we gotta check all the cars and the depot
                if s[i] == 0: # if any car/depot is empty
                    if s[-1]+1 >= tH: # and depot+1 (+1 from the arrival) is at least the threshold 
                        s[-1] += 1 # first add to depot 
                        pickup = min(s[-1], carCap) # then pickup immediately, we can only pick up upto carCap
                        s[-1] -= pickup # decrease the pickup amount from the depot
                        s[i] = pickup # add it to the car
                        return s
                        #break
                    elif s[-1]+1 < tH: # if we cannot exceed the threshold amount to pickup
                        s[-1] += 1 # first add to depot 
                        return s
                                  
            if s[-1] <= depotCap:
                s[-1] += 1
                    
            return s

        elif e > 0: #this is M; drop-off at the nth car
            # there is no condition of a car = 0 here because this cannot happen when there is a drop-off event 
            if 0 < s[e-1] <= carCap: #if the car is not empty or above car cap,
                if tH > s[-1] >= 0: #if there are no customers waiting or we are below the threshold amount
                    s[e-1] = 0 #drop off the customer
                    return s
                elif s[-1] >= tH: #if there are more than the threshold of customers waiting
                    pickUp = min(carCap, s[-1]) #pick up min from car capacity and pickup threshold.
                        #if tH = carCap, then just pick all 
                    s[e-1] = pickUp #put them in the car
                    s[-1] -= pickUp #take them out of the depot
                    return s


### "newList"

Previously defined _new_ function is used to generate the possible transitions of a state $s$ to states $s'_j$ with a given event $e$.

In [7]:
def newList(e, ss):
    
    if ss != None:
        s = ss.copy()
        newList_ = []

        #########
        for check in range(len(s)-1):
            if s[check] > carCap:
                return "car cap"
            elif s[len(s) - 1] > depotCap:
                return "depot cap"
        #########
        if e == 0: #this is Lambda, thus arrival
            for i in range(len(s)): #we gotta check all the cars
                if s[i] == 0: # if any car is empty
                    if s[-1]+1 >= tH: # and depot+1 (+1 from the arrival) is at least the threshold 
                        s[-1] += 1 # add to depot 
                        pickup = min(s[-1], carCap) # then pickup immediately
                        s[-1] -= pickup # decrease the pickup amount from the depot
                        s[i] = pickup # add it to the car
                        newList_.append(s)
                        s = ss.copy()
                        #print("qwe", i, newList_)
                        continue
                    elif s[-1]+1 < tH:
                        s[-1] += 1
                        newList_.append(s)
                        s = ss.copy()
                        #print("asd", i, newList_)
                        return newList_
                else:
                    #print("zxc", i, newList_)
                    continue
                    
            if s[-1] <= depotCap and np.prod(s[:-1]) != 0:
                s[-1] += 1
                if s not in newList_:
                    newList_.append(s)
                #print("rty", i, newList_)
                    
            return newList_

        elif e > 0: #this is M; drop-off at the nth car

            if 0 < s[e-1] <= carCap: #if the car is not empty or above car cap,
                if tH > s[-1] >= 0: #if there are customers waiting and we are below the threshold amount
                    s[e-1] = 0 #drop off the customer
                    newList_.append(s)
                    s = ss.copy()
                    
                elif s[-1] >= tH: #if there are more than the threshold of customers waiting
                    pickUp = min(carCap, s[-1]) #pick up min from car capacity and pickup threshold
                    s[e-1] = pickUp #put them in the car
                    s[-1] -= pickUp #take them out of the depot
                    newList_.append(s)
                    s = ss.copy()
            return newList_


### "rate"

Function find the rate of transition from a state $s$ with a given event $e$.

In [12]:
def rate(e, ss):
    s = ss.copy()
    cSet = Counter(s[:-1]) #counts the numbers in cars from 1 to n, does not consider the depot!!
    
    ########## 
    for check in range(len(s)-1):
        if s[check] > carCap:
            return "car cap"
        elif s[len(s) - 1] > depotCap:
            return "depot cap"
    ##########
    
    
    if e == 0: #this is Lambda
        for i in range(len(s)):
            if s[i] == 0:
                if s[-1]+1 >= tH:
                    if cSet[0] > 0:
                        return L/cSet[0]
                else:
                    return L
        
        return L
        
    elif e > 0: #this is M
        if 0 < s[e - 1] <= carCap: #this is e-1 due to e=n: car n drop-off, and last item being the depot
            #return str(cSet[s[e-1]]) + "M" + str(s[e-1])
            return M[s[e - 1]-1] #try adding all the rates together, and change the output so that its numeric.. 
        else:
            return 0
                


### "rateTot"

Finds the total transition rate, using function *rate*, from state $s$ to $s'$. 

In [15]:
def rateTot(s1, s2):
    
    es = eventspace(len(s1)-1)
    rateTotal = 0
    
    cSet1 = Counter(s1[:-1]) #not reaching the depot here
    
    while es:
        if (s1 != None) and (s2 != None):
            e = es.pop(0) #for e in es:
            for ss2 in newList(e, s1):
                if s2 == ss2:
                    if sum(s2) - sum(s1) == 1: #and np.prod(s1[:-1])==0:
                        rateTotal += rate(e, s1) 
                        #return 222
                    else:
                        rateTotal += rate(e, s1)
                        #print("Asd")
                        #return 333

            
    return rateTotal


1

### "buildRS algorithm"

State-space finding algorithm uses an event space and an initial state (empty system by default) to generate the list of all possible states in the system.

Ciardo, Gianfranco. "Tools for formulating Markov models." Computational Probability. Springer, Boston, MA, 2000. 11-41.

In [11]:
#depotCap = 3
#tH = 1
def buildRS(i0, EventSpace):

    StateSpace = [] #states explored so far
    toExplore = [i0] #states to explore
    
    while toExplore: #while there are more states to explore
        ix = toExplore.pop() 
        StateSpace.append(ix)
        
        for e in EventSpace:
            #print(e)
            if active(e, ix): #if there is an active pass to next state
                j = new(e, ix)
                if j not in StateSpace+toExplore:
                    toExplore.append(j)
                    
    return StateSpace


43

### "augment_Mat" and "get_steady_state"

_augment_Mat_ : is used for augmenting a square matrix, replacing the last row with a vector of ones. 

_get_steady_state_ : is used to linearly solve the system of equations to calculate the stationary distribution for a generator matrix. 

In [18]:
def augment_Mat(Mat):
    dim = Mat.shape[0]
    M = np.vstack((Mat.transpose()[:-1], np.ones(dim)))
    b = np.vstack((np.zeros((dim - 1, 1)), [1]))
    return M, b

def get_steady_state(Mat):
    M, b = augment_Mat(Mat)
    return np.linalg.solve(M, b).transpose()[0]

### Output

#### "iterList"

We can pre-determine the $\lambda$s to use in our stationary distribution calculations.

In [1]:
scaled = 0
scaler = carN
iterList = []
bas = 10
ex = 2
div = 2
if scaled == 1:
    for i in range(bas*ex + 1):
        tt = i/div *scaler
        iterList.append(tt)
else: 
    for i in range(bas*ex + 1):
        tt = i/div 
        iterList.append(tt)
iterList[1:];

NameError: name 'carN' is not defined

#### Singular outputs

Below cell generates the stationary distribution of a CTMC model for a **unique, singular** value of $\lambda$

In [20]:
# globals
carN = 2 # total number of cars
depotCap = 3 # max customers in the depot
carCap = 3 # max customers that can be in a car at any given time
tH = 3 # at what number, does the car start to pick up from depot.
        # tH = 0: no wait, immediate pickup
        # tH > 0: wait until tH people are in the depot. 

        
M = [0.75, 0.44830504827348755, 0.3469597134667903] #
L = 1 # lambda value
stationaryDistributionList = []  


matSize = ((carCap+1) ** carN) * (depotCap+1)
s1 = [0 for i in range(carN + 1)] #initial state
e1 = eventspace(carN) #event space
g = buildRS(s1, e1)

ourMat = np.zeros((len(g), len(g)))

for i in range(len(g)):
    for j in range(len(g)):
        ourMat[i][j] = rateTot(g[i], g[j])

for i in range(len(g)):
    ourMat[i][i] -= np.sum(ourMat[i])

df = pd.DataFrame(data = ourMat[0:, 0:], index = [str(i) for i in g], columns = [str(i) for i in g])

#df.to_csv(r'C:\Users\bzk0054\Desktop\OneDrive - Auburn University\research\TS_Reviews_Edits\edit 2\something.csv', index = True, header=True)
df;

#### Multiple outputs

Below cell generates the stationary distribution of a CTMC model for **multiple** values of $\lambda$, defined within the list _iterList[1:]_

In [ ]:
now = datetime.now()
print(now)

#DO NOT CHANGE
carCap = 3 # max customers that can be in a car at any given time 
#DO NOT CHANGE

# globals
carN = 6 # total number of cars 
depotCap = 3 # max customers in the depot #CHANGEABLE
tH = 1 # at what number, does the car start to pick up from depot.
        # tH = 1: no wait, immediate pickup when at least 1 person is waiting
        # tH > 1: wait until tH people are in the depot waiting, to pick up.
        
scaled = 1
scaler = carN
iterList = []
bas = 10
ex = 2
div = 2
if scaled == 1:
    for i in range(bas*ex + 1):
        tt = i/div *scaler
        iterList.append(tt)
else: 
    for i in range(bas*ex + 1):
        tt = i/div 
        iterList.append(tt)
        
M = [0.75, 0.44830504827348755, 0.3469597134667903] #
stationaryDistributionList = []   
matSize = ((carCap+1) ** carN) * (depotCap+1)
s1 = [0 for i in range(carN + 1)] #initial state
e1 = eventspace(carN) #event space
g = buildRS(s1, e1)

for L in iterList[1:]:
    ourMat = np.zeros((len(g), len(g)))

    for i in range(len(g)):
        for j in range(len(g)):
            ourMat[i][j] = rateTot(g[i], g[j])

    for i in range(len(g)):
        ourMat[i][i] = ourMat[i][i] - np.sum(ourMat[i])

    inTo = get_steady_state(ourMat)
    stationaryDistributionList.append(inTo)

df = pd.DataFrame(data = [i for i in stationaryDistributionList], index = [i for i in iterList[1:]], columns = [str(i) for i in g])

if scaled == 0:
    df.to_csv(r'stuff\sDistList_'+str(carN)+'Cars_'+str(depotCap)+'DepotCap_'+str(tH)+'Threshold.csv', index = True, header=True)
else:
    df.to_csv(r'stuff\sDistList_'+str(carN)+'Cars_'+str(depotCap)+'DepotCap_'+str(tH)+'Threshold_'+str(scaler)+'Scaled.csv', index = True, header=True)
    
w=[] # waiting states
c=[] # car1 states

for i in range(carCap + 1):        
    c.append([j+1 for j, k in enumerate(g) if k[0] == i]) # car1 with i customers in, corresponding to nStatesiinCar1 in Mathematica

for i in range(depotCap + 1):      
    w.append([j+1 for j, k in enumerate(g) if k[-1] == i]) # depot with i customers in, corresponding to nStatesiWait in Mathematica

for k in range(len(w)):
    f = open(str(carN)+"carStates_"+str(depotCap)+"Kap_"+str(tH)+"tH_"+str(k)+"Wait.txt","w")
    f.write("\n".join(str(j) for j in w[k]))
    f.close()

for l in range(len(c)):
    g = open(str(carN)+"carStates_"+str(depotCap)+"Kap_"+str(tH)+"tH_"+str(l)+"Num.txt","w")
    g.write("\n".join(str(j) for j in c[l]))
    g.close()

timing = datetime.now() - now
print(timing)

2021-03-09 11:04:25.399907
